In [1]:
from os import makedirs

import numpy as np
from IPython.display import display
from nibabel import save
from nilearn import image, plotting, reporting
from nimare import correct, io, meta
from scipy.stats import norm

# %% [markdown]
# Because we want to perform the jackknife analysis for multiple ALEs (i.e., the main analysis and the three task category-specific analyses), let's again define a helper function. This takes as its input a Sleuth text file (the same that we've created in Notebook #01 to run the original ALE analysis) plus a couple of additional parameters that will be used for all ALE simulations. These, of course, should be the same as for the original analysis so that the jackknife results will be meaningful.
#
# The logic of our function is to read the Sleuth file and re-create the original ALE analysis (with all experiments inlcuded). Then, it loops over all those experiments and, at each iteration, drops the current one from the sample and re-estimates the ALE. The resulting meta-analytic map is converted into a binary mask telling us which voxels remained statistically significant ($0$ = not signficant, $1$ = significant). Once we've done this for all experiments, the images are averaged into a "jackknife map." This simply shows us for each cluster the percentage of simulations in which it has remained significant. It can therefore be seen as an indicator for the robustness of the cluster against spurious results in the meta-analytic sample.


In [2]:

# Define function to perform a single jackknife analysis
def compute_jackknife(
    text_file="peaks.txt", # 包含元分析峰值数据的slueth文件
    space="mni152_2mm", # 还是使用mni152_2mm空间
    voxel_thresh=0.001,
    cluster_thresh=0.01,
    n_iters=1000,
    random_seed=None,
    output_dir="./",
):

    # Set random seeds if requested
    if random_seed:
        np.random.seed(random_seed)

    # Create NiMARE data set from the Sleuth file
    dset_orig = io.convert_sleuth_to_dataset(text_file)
    study_ids = dset_orig.ids

    # Specify ALE and FWE transformers
    ale = meta.cbma.ALE()
    corr = correct.FWECorrector(
        method="montecarlo", voxel_thresh=voxel_thresh, n_iters=n_iters
    )

    # Create output folder
    _ = makedirs(output_dir, exist_ok=True)

    # Create empty list to store the jackknife'd cluster images
    # 初始化空列表imgs_jk，用于存储每次jackknife的生成的聚类图像
    imgs_jk = []

    for study_id in study_ids:

        # Create new data set with the current study removed
        # 移除当前实验study_id，以便生成新的实验ID列表study_ids_jk  
        study_ids_jk = study_ids[study_ids != study_id]

        # Fit the jackknife'd ALE
        # 创建仅包含study_ids_jk的数据集dset_jk，然后拟合ALE模型
        dset_jk = dset_orig.slice(study_ids_jk)
        res_jk = ale.fit(dset_jk)
        # 并应用FWE校正
        cres_jk = corr.transform(res_jk)

        # Create and save the thresholded cluster mask
        # 生成阈值化的聚类掩模
        img_jk = cres_jk.get_map("z_level-cluster_corr-FWE_method-montecarlo")
        cluster_thresh_z = norm.ppf(1 - cluster_thresh / 2)
        formula = "np.where(img > " + str(cluster_thresh_z) + ", 1, 0)"
        img_jk = image.math_img(formula, img=img_jk)

        # Save to the output folder and to our list
        study_id_short = study_id.replace("-", "")
        save(img_jk, filename=output_dir + "/jk_" + study_id_short + ".nii.gz")
        imgs_jk.append(img_jk)

    # Create and save averaged jackknife image
    img_mean = image.mean_img(imgs_jk)
    save(img_mean, filename=output_dir + "/mean_jk.nii.gz")

    return img_mean

In [10]:

# So far we've only defined the jackknife function; now let's apply it to our ALE analyses. We just need to list the Sleuth text file names and provide our default ALE thresholds and settings. The function will run for a couple of hours and return, for each input Sleuth file, an averaged jackknife map.

# List the Sleuth files for which to run a jackknife analysis
# 生成包含所有实验的Sleuth文件
# prefixes = ["all", "knowledge", "relatedness", "objects"]
# text_files = ["../results/ale/" + prefix + ".txt" for prefix in prefixes]

# # Create output directory names
# output_dirs = ["../results/jackknife/" + prefix for prefix in prefixes]

text_file = "/Users/ss/Documents/Psych_ALE_meta/data/Self_all.txt"

output_dir = "../results/jackknife/"

In [12]:
# Apply the jackknife function

jk = compute_jackknife(  
    text_file=text_file,  
    space="mni152_2mm",  
    voxel_thresh=0.001,  
    cluster_thresh=0.05,  
    n_iters=5000,  
    random_seed=1234,  
    output_dir=output_dir,  
)  


INFO:nimare.correct:Using correction method implemented in Estimator: nimare.meta.cbma.ale.ALE.correct_fwe_montecarlo.


  0%|          | 0/5000 [00:00<?, ?it/s]

INFO:nimare.meta.cbma.base:Using null distribution for voxel-level FWE correction.


ValueError: No map with name 'z_level-cluster_corr-FWE_method-montecarlo' found.

In [ ]:

# Let's examine one of these maps, here for our main analysis containing all semantic knowledge experiments. Note that most clusters have a jackknife reliability of $1.0$, indicating strong robustness against the deletion of individual experiments. Only for two of the posterior clusters in the right hemisphere, the robustness is slightly reduced (but still high at approx. $0.85$).

# Glass brain example
img_jk_all = image.load_img("../results/jackknife/all/mean_jk.nii.gz")
p = plotting.plot_glass_brain(None, display_mode="lyrz", colorbar=True)
p.add_overlay(img_jk_all, colorbar=True, cmap="RdYlGn", vmin=0, vmax=1)


In [ ]:

# Cluster table example
t = reporting.get_clusters_table(img_jk_all, stat_threshold=0, min_distance=1000)
display(t)
